In [ ]:

!wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
# !apt-get install software-properties-common
# !add-apt-repository ppa:fenics-packages/fenics
# !apt-get update -qq
# !apt-get install fenics

--2024-01-08 20:15:09--  https://fem-on-colab.github.io/releases/fenics-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.06K  --.-KB/s    in 0s      

2024-01-08 20:15:09 (41.5 MB/s) - ‘/tmp/fenics-install.sh’ saved [4156/4156]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ set +x
























################################################################################
#     This 

In [ ]:
from fenics import *
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
set_log_level(30)


In [ ]:
# Create mesh and define function space
# mesh = UnitSquareMesh(32, 32)
space_size = 50  # For example, space size = 1.0
p0 = Point(0.0, 0.0)
p1 = Point(space_size,space_size)
mesh = RectangleMesh(p0, p1, 50, 50,"right/left")

V = FunctionSpace(mesh, 'P', 1)

In [ ]:
# Define functions for c, s, and i
c = Function(V)
s = Function(V)
i = Function(V)
phi_c = TestFunction(V)
phi_s = TestFunction(V)
phi_i = TestFunction(V)


In [ ]:
# Define constants alpha and beta
D_c = Constant(0.01)
D_s = Constant(0.09)
D_i = Constant(0.05)
ri = Constant(0.595)
rs = Constant(0.6)
gamma = Constant(0.054)
delta = Constant(0.012)
alpha = Constant(0.33)
beta = Constant(0.06)
alle = Constant(1)
eta = Constant(0.365)

In [ ]:

# Define time step and time interval
T = 1   # final time
num_steps = 100  # number of time steps
dt = T / num_steps  # time step size
print(dt)

0.01


In [ ]:
c_n = interpolate(Expression("rand()", degree=2), V)
s_n = interpolate(Expression("rand()", degree=2), V)
i_n = interpolate(Expression("rand()", degree=2), V)

In [ ]:
# Define the weak forms for the equations with time dependence
F_c = ((c - c_n) / dt) * phi_c * dx + D_c * dot(grad(c), grad(phi_c)) * dx + \
      c * (c - alle) * (1 - c) * phi_c * dx - alpha * c * s * phi_c * dx - beta * i * c * phi_c * dx

F_s = ((s - s_n) / dt) * phi_s * dx + D_s * dot(grad(s), grad(phi_s)) * dx + \
      s * (1 - s) * (s - gamma * c) * phi_s * dx - delta * s * i * phi_s * dx

F_i = ((i - i_n) / dt) * phi_i * dx + D_i * dot(grad(i), grad(phi_i)) * dx + \
      i * (1 - i) * (i - eta * c * i) * phi_i * dx

In [ ]:
# Define functions for plotting
c_plot = Function(V)
s_plot = Function(V)
i_plot = Function(V)

In [ ]:
# Time-stepping
t = 0

In [ ]:
while (t < T):
    print(t)
    t += dt

    # Solve the system for the next time step
    solve(F_c == 0, c)
    solve(F_s == 0, s)
    solve(F_i == 0, i)

    # Update previous solution
    c_n.assign(c)
    s_n.assign(s)
    i_n.assign(i)

    # Plot the solutions at each time step
    c_plot.assign(c_n)
    s_plot.assign(s_n)
    i_plot.assign(i_n)


    plt.figure(figsize=(20, 10))

    plt.subplot(1, 3, 1)
    p1 = plot(c_plot)
    p1.set_cmap("seismic")
    plt.title(f'Solution for c at time {t:.2f}')
    aspect = 20
    pad_fraction = 0.5
    ax = plt.gca()
    divider = make_axes_locatable(ax)
    width = axes_size.AxesY(ax, aspect=1./aspect)
    pad = axes_size.Fraction(pad_fraction, width)
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.colorbar(p1, cax=cax)


    plt.subplot(1, 3, 2)
    p2 = plot(s_plot)
    p2.set_cmap("gray")
    plt.title(f'Solution for s at time {t:.2f}')
    aspect = 20
    pad_fraction = 0.5
    ax = plt.gca()
    divider = make_axes_locatable(ax)
    width = axes_size.AxesY(ax, aspect=1./aspect)
    pad = axes_size.Fraction(pad_fraction, width)
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.colorbar(p2, cax=cax)

    plt.subplot(1, 3, 3)
    p3 = plot(i_plot)
    p3.set_cmap("gray")
    plt.title(f'Solution for i at time {t:.2f}')
    aspect = 20
    pad_fraction = 0.5
    ax = plt.gca()
    divider = make_axes_locatable(ax)
    width = axes_size.AxesY(ax, aspect=1./aspect)
    pad = axes_size.Fraction(pad_fraction, width)
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.colorbar(p3, cax=cax)

    plt.tight_layout(pad=4)
    plt.show()


Output hidden; open in https://colab.research.google.com to view.